In [1]:
import threading

# Создание потоков

In [2]:
def foo(n, thread_num):
    res = 1
    for i in range(1, n + 1):
        res *= i
    
    print('Thread {}: {}\n'.format(thread_num, res))

thread1 = threading.Thread(target=foo, args=(5, 1))
thread2 = threading.Thread(target=foo, args=(100, 2))

thread1.start()
thread2.start()

thread1.join()
thread2.join()

Thread 1: 120

Thread 2: 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000



# ThreadPoolExecutor

Создание поток - дорогая операция. `ThreadPoolExecutor` - это пул поток, методв `submit` назначает заданию какой-то свободный поток (если есть свободные, иначе ставит в очередь). 

`Future` (оно же в определенном варианте `Promise`, `CompletableFuture`) - это некоторая ручка, которая позволяет отслеживать процесс выполнения задания, получать результат и т.п.

In [3]:
from concurrent.futures import ThreadPoolExecutor, Future
from tqdm import tqdm

def fact(n):
    res = 1
    for i in range(1, n + 1):
        res *= i
    
    return res

def add(a, b):
    return a + b

executor = ThreadPoolExecutor(max_workers=3)

future1 = executor.submit(fact, 5)
future2 = executor.submit(fact, 10)
future3 = executor.submit(add, 5, 3)

future3.result(), future1.result(), future2.result()

(8, 120, 3628800)

Есть многопоточный аналог функции `map`

In [4]:
it = [x * 10 for x in range(10000)]

[x**2 for x in it]
map(lambda x: x**2, it)

In [5]:
it = [x * 10 for x in range(10000)]
for r in executor.map(lambda x: x ** 2, it):
    pass

# Блокировки

`Lock` - самый просто примитив синхронизации. Для блокировки можно (нужно) использовать котнекстные менеджеры

In [6]:
lock = threading.Lock()

# взять блокировку
lock.acquire()
try:
    # do something
    pass
finally:
    # не забываем освободить
    lock.release()
    
    
# или тоже самое по смыслу
with lock:
    # do something
    pass

Когда имеем дело с общими для потоко данными - блокировка обязательна, даже для `Python`, в котором есть `GIL`. В данном примере каждый поток увеличивает счетчик. 

In [7]:
lock_a = threading.Lock()
lock_b = threading.Lock()

a = 0
b = 0

def func_a():
    with lock_a:
        with lock_b:
            a = 5
            b = 1


def func_b():
    with lock_a:
        with lock_b:
            b = 2
            a = 1

executor.submit(func_a)
executor.submit(func_b)

<Future at 0x7f6701d48b38 state=pending>

In [8]:
executor = ThreadPoolExecutor(max_workers=3)

lock = threading.Lock()
a = 1

def func():
    global a
    with lock:
       a += 1

for idx in range(100000):
    executor.submit(func)

In [9]:
acc = 0

lock = threading.Lock()

def foo(lock):    
    global acc
    
    with lock:
        acc += 1
            
        
futures = [executor.submit(foo, lock) for _ in range(10)]
for future in futures:
    future.result()

# Очереди

Для взаимодействия потоков можно использовать очереди. Очереди поддерживают две операции:
   - положить в очередь `put()`
   - взять из очереди `get()`
   
Очерель может иметь фиксированный размер, в тако случае операция `put()` блокируется, если очередь полна, а операция `get()` блокируется если очередь пуста.

In [10]:
import requests
import queue
import lxml.html

num_workers = 3

# функция получает сообщения из очереди `q`. Если там ссылка - загружает сайт по ссылке. Если `None` - завершает свое выполнение
def worker(q: queue.Queue):
    titles = []
    
    while True:
        item = q.get()
        if item is None:
            break        
        res = requests.get(item)
        titles.append(lxml.html.fromstring(res.text).xpath('//title/text()')[0].strip())
    
    return ';'.join(titles)
        
# Создаем очередь
q = queue.Queue(maxsize=2)

# создем потоки (для простоты засовываем всё в Executor)
futures = [executor.submit(worker, q) for _ in range(num_workers)]
    
# добавляем ссылки в очередь    
for item in ['http://fontanka.ru', 'http://lenta.ru', 'http://gazeta.ru']:
    q.put(item)

# добавляем в очередь None, чтобы завершить все потоки    
for i in range(num_workers):
    q.put(None)

    
for t in futures:
    print(t.result())

Новости Санкт-Петербурга - главные новости сегодня | fontanka.ru - новости Санкт-Петербурга
Lenta.ru - Новости России и мира сегодня
Главные новости - Газета.Ru


# Процессы

Потоки в `CPython` неэффективны для вычислительно-сложный операций, так как присутствут `GIL`. В качестве альтернативы есть модуль `multiprocessing`, который создает процессы вместо потоков. Новый процесс имеет своё адресное пространство памяти, потому взаимодействие между процессами некоторым образом осложнено. Более того, метод создание дочерних процессов может различаться в зависимости от операционной системы.

In [11]:
# Загружаем параллельно три ссылки

from multiprocessing import Process, Pool

import time

def fact(n):
    res = 1
    for i in range(1, n):
        res *= i
    
    return res

with Pool() as pool:    
    fut1 = pool.apply_async(fact, args=(100,))
    fut2 = pool.apply_async(fact, args=(200,))
    print(fut1.get(), fut2.get())

933262154439441526816992388562667004907159682643816214685929638952175999932299156089414639761565182862536979208272237582511852109168640000000000000000000000 3943289336823952517761816069660925311475679888435866316473712666221797249817016714601521420059923119520886060694598194151288213951213185525309633124764149655567314286353816586186984944719612228107258321201270166459320656137141474266387621212037869516201606287027897843301130159520851620311758504293980894611113948118519486873600000000000000000000000000000000000000000000000


In [12]:
# Загружаем параллельно три ссылки

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

import time

def fact(n):
    res = 1
    for i in range(1, n):
        res *= i
    
    return res

with ProcessPoolExecutor(4) as pool:
    fut1 = pool.submit(fact, 100)
    fut2 = pool.submit(fact, 200)
    print(fut1.result(), fut2.result())

933262154439441526816992388562667004907159682643816214685929638952175999932299156089414639761565182862536979208272237582511852109168640000000000000000000000 3943289336823952517761816069660925311475679888435866316473712666221797249817016714601521420059923119520886060694598194151288213951213185525309633124764149655567314286353816586186984944719612228107258321201270166459320656137141474266387621212037869516201606287027897843301130159520851620311758504293980894611113948118519486873600000000000000000000000000000000000000000000000
